In [1]:
# Gerador de Testes Unitários Automático
# Protótipo para o Projeto Integrado de GenAI em Engenharia de Software
# Autor: Luis Roberto Lange
!pip install openai==0.28
import os
import re
import ast
import json
import inspect
import textwrap
from IPython.display import Markdown, display
import requests
import numpy as np
import pandas as pd
from datetime import datetime
import openai
from google.colab import userdata, files

# Configuração da API da OpenAI
# Recupera a API key de forma segura
try:
    openai.api_key = userdata.get('OPENAI_API_KEY')
except:
    # Solicitar chave da API caso não esteja nas variáveis de ambiente
    print("Por favor, configure sua API key da OpenAI para continuar.")
    openai.api_key = input("Digite sua API key da OpenAI: ")

class CodeAnalyzer:
    """Classe responsável por analisar o código fonte e extrair informações relevantes"""

    def __init__(self, code_string):
        self.code_string = code_string
        self.parsed_ast = None
        try:
            self.parsed_ast = ast.parse(code_string)
        except SyntaxError as e:
            print(f"Erro de sintaxe no código: {e}")

    def extract_functions_and_classes(self):
        """Extrai funções e classes do código fonte"""
        if not self.parsed_ast:
            return [], []

        functions = []
        classes = []

        for node in ast.walk(self.parsed_ast):
            # Extrai funções
            if isinstance(node, ast.FunctionDef):
                functions.append({
                    'name': node.name,
                    'args': [arg.arg for arg in node.args.args],
                    'docstring': ast.get_docstring(node) or '',
                    'code': self._get_source_segment(node)
                })

            # Extrai classes
            elif isinstance(node, ast.ClassDef):
                methods = []
                for item in node.body:
                    if isinstance(item, ast.FunctionDef):
                        methods.append({
                            'name': item.name,
                            'args': [arg.arg for arg in item.args.args],
                            'docstring': ast.get_docstring(item) or '',
                            'code': self._get_source_segment(item)
                        })

                classes.append({
                    'name': node.name,
                    'docstring': ast.get_docstring(node) or '',
                    'methods': methods,
                    'code': self._get_source_segment(node)
                })

        return functions, classes

    def _get_source_segment(self, node):
        """Extrai o segmento de código fonte para um nó AST"""
        try:
            start_line = node.lineno - 1
            end_line = node.end_lineno
            lines = self.code_string.splitlines()
            return '\n'.join(lines[start_line:end_line])
        except:
            # Fallback para quando lineno/end_lineno não está disponível
            return "<Código não disponível>"

    def extract_imports(self):
        """Extrai todas as importações do código fonte"""
        if not self.parsed_ast:
            return []

        imports = []
        for node in ast.walk(self.parsed_ast):
            if isinstance(node, ast.Import):
                for name in node.names:
                    imports.append(name.name)
            elif isinstance(node, ast.ImportFrom):
                module = node.module or ''
                for name in node.names:
                    imports.append(f"{module}.{name.name}")

        return imports

class TestGenerator:
    """Classe responsável por gerar testes unitários usando IA Generativa"""

    def __init__(self, model="gpt-4o-mini"):
        self.model = model

    def generate_tests(self, code_info, language="python", framework="pytest"):
        """
        Gera testes unitários com base nas informações extraídas do código

        Args:
            code_info: Dicionário com funções, classes e imports extraídos
            language: Linguagem de programação (python, javascript, java)
            framework: Framework de testes (pytest, jest, junit)

        Returns:
            str: Código dos testes unitários gerados
        """
        # Monta o prompt para o modelo de IA
        prompt = self._build_prompt(code_info, language, framework)

        #print(prompt)

        # Chama a API da OpenAI
        try:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "Você é um especialista em engenharia de software focado na geração de testes unitários de alta qualidade. Seu objetivo é analisar o código fornecido e criar testes unitários eficazes que garantam a correta funcionalidade e cobertura do código."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2, # Valor baixo para gerar respostas mais determinísticas
                max_tokens=4000
            )

            return response.choices[0].message['content']
        except Exception as e:
            return f"Erro ao gerar testes: {str(e)}"

    def _build_prompt(self, code_info, language, framework):
        """Constrói o prompt para o modelo de IA"""
        functions_str = json.dumps(code_info['functions'], indent=2) if 'functions' in code_info else "[]"
        classes_str = json.dumps(code_info['classes'], indent=2) if 'classes' in code_info else "[]"
        imports_str = json.dumps(code_info['imports'], indent=2) if 'imports' in code_info else "[]"

        prompt = f"""
        Por favor, gere testes unitários para o seguinte código {language} usando o framework {framework}.

        # Importações do código:
        {imports_str}

        # Funções no código:
        {functions_str}

        # Classes no código:
        {classes_str}

        Requisitos para os testes:
        1. O teste deve cobrir todos os caminhos de execução possíveis
        2. Deve incluir casos de teste para condições de borda
        3. Deve testar tanto casos de sucesso quanto casos de falha
        4. Deve seguir as melhores práticas para testes unitários em {framework}
        5. Deve ter uma cobertura de código alta
        6. O código de teste deve ser claro e bem comentado
        7. Inclua docstrings apropriados explicando o que cada teste faz

        Por favor, gere apenas o código do teste, sem explicações adicionais.
        """

        return prompt

class TestEvaluator:
    """Classe responsável por avaliar a qualidade dos testes gerados"""

    def __init__(self):
        pass

    def evaluate_tests(self, code, tests, language="python"):
        """
        Avalia a qualidade dos testes gerados

        Args:
            code: Código fonte original
            tests: Testes gerados
            language: Linguagem de programação

        Returns:
            dict: Avaliação dos testes gerados
        """
        # Para este protótipo, usamos a IA para avaliar a qualidade dos testes
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "Você é um especialista em qualidade de software focado em avaliar testes unitários. Sua tarefa é analisar o código de teste e fornecer métricas de qualidade."},
                    {"role": "user", "content": f"""
                    Avalie a qualidade dos testes unitários abaixo com base no código fonte.

                    # Código fonte original:
                    ```{language}
                    {code}
                    ```

                    # Testes gerados:
                    ```{language}
                    {tests}
                    ```

                    Forneça uma avaliação quantitativa (de 0 a 10) para os seguintes critérios:
                    1. Cobertura de código estimada
                    2. Qualidade dos casos de teste
                    3. Testes de casos de borda
                    4. Clareza e legibilidade
                    5. Adequação ao framework de testes

                    Responda usando apenas um objeto JSON com esses campos, sem texto adicional.
                    """}
                ],
                temperature=0.2
            )

            # Extrair o JSON da resposta
            content = response.choices[0].message['content']
            # Procurar por um bloco JSON na resposta
            json_match = re.search(r'\{.*\}', content, re.DOTALL)
            if json_match:
                try:
                    result = json.loads(json_match.group(0))
                    return result
                except:
                    return {"error": "Falha ao processar o JSON da avaliação"}
            else:
                return {"error": "Resposta de avaliação não contém JSON válido"}

        except Exception as e:
            return {"error": f"Erro ao avaliar testes: {str(e)}"}

def upload_and_analyze_code():
    """Função para fazer upload e analisar um arquivo de código"""
    print("Por favor, faça upload do arquivo de código:")
    uploaded = files.upload()

    file_name = list(uploaded.keys())[0]
    code_content = uploaded[file_name].decode('utf-8')

    # Limpar o nome do arquivo - remover números entre parênteses
    clean_file_name = re.sub(r' \(\d+\)', '', file_name)

    analyzer = CodeAnalyzer(code_content)
    functions, classes = analyzer.extract_functions_and_classes()
    imports = analyzer.extract_imports()

    code_info = {
        'functions': functions,
        'classes': classes,
        'imports': imports
    }

    # Identificar a linguagem com base na extensão do arquivo
    language = "python"  # Padrão
    if clean_file_name.endswith(".js"):
        language = "javascript"
    elif clean_file_name.endswith(".java"):
        language = "java"

    # Identificar o framework de testes baseado na linguagem
    framework = "pytest"  # Padrão para Python
    if language == "javascript":
        framework = "jest"
    elif language == "java":
        framework = "junit"

    # Gerar testes
    generator = TestGenerator()
    print(f"Gerando testes unitários para {clean_file_name} usando {framework}...")

    tests = generator.generate_tests(code_info, language, framework)

    # Avaliar os testes gerados
    evaluator = TestEvaluator()
    evaluation = evaluator.evaluate_tests(code_content, tests, language)

    # Mostrar resultados
    display(Markdown(f"## Testes gerados para {clean_file_name}"))
    display(Markdown(f"```{language}\n{tests}\n```"))

    display(Markdown("## Avaliação dos testes"))
    if "error" in evaluation:
        display(Markdown(f"Erro na avaliação: {evaluation['error']}"))
    else:
        for key, value in evaluation.items():
            display(Markdown(f"- **{key}**: {value}/10"))

    # Criar o conteúdo da avaliação em formato Markdown
    evaluation_md = "# Avaliação dos Testes Unitários\n\n"
    evaluation_md += f"**Arquivo:** {clean_file_name}\n"
    evaluation_md += f"**Framework:** {framework}\n"
    evaluation_md += f"**Data:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
    evaluation_md += "## Métricas de Qualidade\n\n"

    if "error" in evaluation:
        evaluation_md += f"**Erro na avaliação:** {evaluation['error']}\n"
    else:
        for key, value in evaluation.items():
            evaluation_md += f"- **{key}**: {value}/10\n"

    # Opção 1: Incluir a avaliação como comentários no início do arquivo de teste
    file_base_name, file_extension = os.path.splitext(clean_file_name)
    test_file_name = f"Unit_Test_{file_base_name}{file_extension}"

    # Preparar comentários com base na linguagem
    comment_start = "# " if language == "python" else "// " if language in ["javascript", "java"] else "# "

    evaluation_comment = f"{comment_start}Avaliação dos Testes Unitários para {clean_file_name}\n"
    if "error" not in evaluation:
        for key, value in evaluation.items():
            evaluation_comment += f"{comment_start}{key}: {value}/10\n"
    evaluation_comment += f"{comment_start}Gerado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"

    # Salvar o arquivo de teste com a avaliação no início
    with open(test_file_name, 'w') as f:
        f.write(evaluation_comment)
        f.write(tests)

    # Opção 2: Salvar um arquivo markdown separado com a avaliação
    evaluation_file_name = f"Avaliacao_{file_base_name}_{framework}.md"
    with open(evaluation_file_name, 'w') as f:
        f.write(evaluation_md)

    # Fazer download dos arquivos gerados
    files.download(test_file_name)
    files.download(evaluation_file_name)

    print(f"Testes salvos em {test_file_name}")
    print(f"Avaliação salva em {evaluation_file_name}")

    return code_content, tests, evaluation

# Interface principal para o Colab
def main():
    display(Markdown("# Gerador de Testes Unitários Automático"))
    display(Markdown("""
    Esta ferramenta analisa seu código e gera testes unitários utilizando IA Generativa.

    ## Como usar:
    1. Faça upload de um arquivo de código
    2. O sistema analisará o código e gerará testes unitários
    3. Os testes serão avaliados quanto à sua qualidade
    4. Você poderá baixar os testes gerados

    ## Linguagens e frameworks suportados:
    - Python (pytest)
    - JavaScript (Jest)
    - Java (JUnit)
    """))

    upload_and_analyze_code()

# Executar a aplicação
if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.70.0
    Uninstalling openai-1.70.0:
      Successfully uninstalled openai-1.70.0


# Gerador de Testes Unitários Automático


    Esta ferramenta analisa seu código e gera testes unitários utilizando IA Generativa.

    ## Como usar:
    1. Faça upload de um arquivo de código
    2. O sistema analisará o código e gerará testes unitários
    3. Os testes serão avaliados quanto à sua qualidade
    4. Você poderá baixar os testes gerados

    ## Linguagens e frameworks suportados:
    - Python (pytest)
    - JavaScript (Jest)
    - Java (JUnit)
    

Por favor, faça upload do arquivo de código:


Saving FinancialCalculator.py to FinancialCalculator.py
Gerando testes unitários para FinancialCalculator.py usando pytest...

        Por favor, gere testes unitários para o seguinte código python usando o framework pytest.

        # Importações do código:
        []

        # Funções no código:
        [
  {
    "name": "calculate_future_value",
    "args": [
      "principal",
      "rate",
      "time_years",
      "compounding_frequency"
    ],
    "docstring": "Calcula o valor futuro de um investimento com juros compostos.\n\nArgs:\n    principal (float): Valor principal (montante inicial)\n    rate (float): Taxa de juros anual (em percentual, ex: 5.5 para 5.5%)\n    time_years (float): Tempo em anos\n    compounding_frequency (str): Frequ\u00eancia de capitaliza\u00e7\u00e3o dos juros:\n                                 \"annual\", \"semiannual\", \"quarterly\", \"monthly\", \"daily\"\n\nReturns:\n    float: Valor futuro do investimento\n\nRaises:\n    ValueError: Se o principa

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Testes salvos em Unit_Test_FinancialCalculator.py
Avaliação salva em Avaliacao_FinancialCalculator_pytest.md
